In [1]:
pip install pandas numpy scikit-learn nltk scipy tensorflow


In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import hstack, csr_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import gc
import nltk
from nltk.tokenize import word_tokenize

# Download NLTK data
nltk.download('punkt')

# Load the dataset
file_path = '/content/PDFMalware2022.csv'
df = pd.read_csv(file_path)

# Handle missing values
# Fill missing numerical values with the median value of each column
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
numerical_cols = numerical_cols.drop('Class')  # Exclude the target column
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].median())

# Fill missing categorical values with the mode value of each column
categorical_cols = df.select_dtypes(include=['object']).columns
df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])

# Check for and handle missing values in the target column
if df['Class'].isnull().any():
    print("Found NaN values in the target column. Filling with mode value.")
    df['Class'] = df['Class'].fillna(df['Class'].mode()[0])

# Encode categorical variables if any exist
# For this dataset, only 'Class' needs to be considered

# Tokenization and Hashing Vectorizer for text columns
text_cols = ['File name', 'text', 'header']
hashing_vectorizer = HashingVectorizer(n_features=2**6, alternate_sign=False)  # Further reduced number of features for efficiency

# Apply tokenization and vectorization
text_features = [hashing_vectorizer.transform(df[col].apply(lambda x: ' '.join(word_tokenize(x)))) for col in text_cols]

# Combine text features into a single sparse matrix
text_features_combined = hstack(text_features)

# Separate numerical features and target
X_numerical = df[numerical_cols].values
y = df['Class'].values

# Standardize numerical features
scaler = StandardScaler()
X_numerical = scaler.fit_transform(X_numerical)

# Combine numerical and text features
X = hstack([csr_matrix(X_numerical), text_features_combined])

# Reduce dimensionality with Truncated SVD
svd = TruncatedSVD(n_components=30, random_state=42)  # Further reduced number of components for efficiency
X_reduced = svd.fit_transform(X)

# Adjust labels for one-hot encoding
y_adjusted = y - 1

# 10-fold cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
classification_reports = []

for fold, (train_index, test_index) in enumerate(skf.split(X_reduced, y)):
    X_train, X_test = X_reduced[train_index], X_reduced[test_index]
    y_train, y_test = y_adjusted[train_index], y_adjusted[test_index]

    print(f"Fold {fold + 1}:")
    print(f"Training data size: {len(train_index)}")
    print(f"Testing data size: {len(test_index)}")

    # Define the neural network model
    model = Sequential([
        Dense(64, input_dim=X_train.shape[1], activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

    # Predict using the trained model
    y_pred_test = model.predict(X_test).flatten()

    # Adjust predicted labels back to original range (1 and 2)
    y_pred_test_labels_adjusted = np.round(y_pred_test).astype(int) + 1
    y_test_labels_adjusted = y_test + 1

    # Evaluate the model
    accuracy = accuracy_score(y_test_labels_adjusted, y_pred_test_labels_adjusted)
    precision = precision_score(y_test_labels_adjusted, y_pred_test_labels_adjusted, average='weighted')
    recall = recall_score(y_test_labels_adjusted, y_pred_test_labels_adjusted, average='weighted')
    f1 = f1_score(y_test_labels_adjusted, y_pred_test_labels_adjusted, average='weighted')
    report = classification_report(y_test_labels_adjusted, y_pred_test_labels_adjusted, output_dict=True)

    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    classification_reports.append(report)

    print(f"Fold {fold + 1} Results:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}\n")

    # Free up memory
    del X_train, X_test, y_train, y_test, model, y_pred_test
    gc.collect()

# Aggregate the classification reports
average_classification_report = {}
for key in classification_reports[0].keys():
    if isinstance(classification_reports[0][key], dict):
        average_classification_report[key] = {}
        for sub_key in classification_reports[0][key].keys():
            average_classification_report[key][sub_key] = np.mean([report[key][sub_key] for report in classification_reports])
    else:
        average_classification_report[key] = np.mean([report[key] for report in classification_reports])

# Calculate average metrics
average_accuracy = np.mean(accuracy_scores)
average_precision = np.mean(precision_scores)
average_recall = np.mean(recall_scores)
average_f1_score = np.mean(f1_scores)

print(f"Average Model Accuracy: {average_accuracy}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1 Score: {average_f1_score}")
print("Average Classification Report:")
for key, value in average_classification_report.items():
    if isinstance(value, dict):
        print(f"  {key}:")
        for sub_key, sub_value in value.items():
            print(f"    {sub_key}: {sub_value}")
    else:
        print(f"  {key}: {value}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Found NaN values in the target column. Filling with mode value.
Fold 1:
Training data size: 9023
Testing data size: 1003
Epoch 1/10
282/282 [==============================] - 4s 5ms/step - loss: 0.5502 - accuracy: 0.7346
Epoch 2/10
282/282 [==============================] - 2s 5ms/step - loss: 0.3983 - accuracy: 0.8463
Epoch 3/10
282/282 [==============================] - 2s 5ms/step - loss: 0.3491 - accuracy: 0.8684
Epoch 4/10
282/282 [==============================] - 3s 9ms/step - loss: 0.3254 - accuracy: 0.8776
Epoch 5/10
282/282 [==============================] - 2s 8ms/step - loss: 0.3037 - accuracy: 0.8833
Epoch 6/10
282/282 [==============================] - 2s 7ms/step - loss: 0.2763 - accuracy: 0.8919
Epoch 7/10
282/282 [==============================] - 1s 5ms/step - loss: 0.2709 - accuracy: 0.8911
Epoch 8/10
282/282 [==============================] - 2s 5ms/step - loss: 0.2524 - accuracy: 0.9067
Epoch 9/10
282/282 [==============================] - 1s 4ms/step - loss: 0.227